In [174]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random 
from sklearn.metrics import classification_report
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [183]:
data=pd.read_csv("/content/email_upd.txt",encoding="latin1",delimiter='\t')
data.head()

,Sentence,Tag
0,Send email to Haris that call me at 5 PM on 12...,O O O B-email.to O B-email.text I-email.text I...
1,I would like to mail to Nisha that there will ...,O O O O O O B-email.to O B-email.text I-email....
2,Mail to Satya that Complete the assignment by ...,O O B-email.to O B-email.text I-email.text I-e...
3,Please email to Lucifer that Please be on time...,O O O B-email.to O B-email.text I-email.text ...
4,Send email to Maverick that we are moving to P...,O O O B-email.to O B-email.text I-email.text I...


In [184]:
data.dropna(inplace=True)

In [185]:
data.shape

(36, 2)

In [186]:
data.head()

,Sentence,Tag
0,Send email to Haris that call me at 5 PM on 12...,O O O B-email.to O B-email.text I-email.text I...
1,I would like to mail to Nisha that there will ...,O O O O O O B-email.to O B-email.text I-email....
2,Mail to Satya that Complete the assignment by ...,O O B-email.to O B-email.text I-email.text I-e...
3,Please email to Lucifer that Please be on time...,O O O B-email.to O B-email.text I-email.text ...
4,Send email to Maverick that we are moving to P...,O O O B-email.to O B-email.text I-email.text I...


In [187]:
agg_data = data.copy(True)
agg_data['tokenised_sentences']=agg_data['Sentence'].apply(lambda x:x.split())
agg_data['tag_list']=agg_data['Tag'].apply(lambda x:x.split())
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list
0,Send email to Haris that call me at 5 PM on 12...,O O O B-email.to O B-email.text I-email.text I...,"[Send, email, to, Haris, that, call, me, at, 5...","[O, O, O, B-email.to, O, B-email.text, I-email..."
1,I would like to mail to Nisha that there will ...,O O O O O O B-email.to O B-email.text I-email....,"[I, would, like, to, mail, to, Nisha, that, th...","[O, O, O, O, O, O, B-email.to, O, B-email.text..."
2,Mail to Satya that Complete the assignment by ...,O O B-email.to O B-email.text I-email.text I-e...,"[Mail, to, Satya, that, Complete, the, assignm...","[O, O, B-email.to, O, B-email.text, I-email.te..."
3,Please email to Lucifer that Please be on time...,O O O B-email.to O B-email.text I-email.text ...,"[Please, email, to, Lucifer, that, Please, be,...","[O, O, O, B-email.to, O, B-email.text, I-email..."
4,Send email to Maverick that we are moving to P...,O O O B-email.to O B-email.text I-email.text I...,"[Send, email, to, Maverick, that, we, are, mov...","[O, O, O, B-email.to, O, B-email.text, I-email..."


In [188]:
agg_data['len_sentence']=agg_data['tokenised_sentences'].apply(lambda x:len(x))
agg_data['len_tag']=agg_data['tag_list'].apply(lambda x:len(x))
agg_data['is_equal']=agg_data.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
agg_data['is_equal'].value_counts()

1    36
Name: is_equal, dtype: int64

In [189]:
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal
0,Send email to Haris that call me at 5 PM on 12...,O O O B-email.to O B-email.text I-email.text I...,"[Send, email, to, Haris, that, call, me, at, 5...","[O, O, O, B-email.to, O, B-email.text, I-email...",13,13,1
1,I would like to mail to Nisha that there will ...,O O O O O O B-email.to O B-email.text I-email....,"[I, would, like, to, mail, to, Nisha, that, th...","[O, O, O, O, O, O, B-email.to, O, B-email.text...",15,15,1
2,Mail to Satya that Complete the assignment by ...,O O B-email.to O B-email.text I-email.text I-e...,"[Mail, to, Satya, that, Complete, the, assignm...","[O, O, B-email.to, O, B-email.text, I-email.te...",10,10,1
3,Please email to Lucifer that Please be on time...,O O O B-email.to O B-email.text I-email.text ...,"[Please, email, to, Lucifer, that, Please, be,...","[O, O, O, B-email.to, O, B-email.text, I-email...",11,11,1
4,Send email to Maverick that we are moving to P...,O O O B-email.to O B-email.text I-email.text I...,"[Send, email, to, Maverick, that, we, are, mov...","[O, O, O, B-email.to, O, B-email.text, I-email...",12,12,1


In [190]:
agg_data[agg_data['is_equal']==0].head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal


In [191]:
agg_data=agg_data[agg_data['is_equal']!=0]

In [192]:
agg_data.shape

(36, 7)

In [193]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  36
Are number of Sentences and Tag list equal  True


In [194]:
sentences_list[0]

'Send email to Haris that call me at 5 PM on 12th October'

In [195]:
tags_list[0]

['O',
 'O',
 'O',
 'B-email.to',
 'O',
 'B-email.text',
 'I-email.text',
 'I-email.text',
 'I-email.text',
 'I-email.text',
 'I-email.text',
 'I-email.text',
 'I-email.text']

In [196]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentences_list)

In [197]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  194


In [198]:
tokeniser.index_word[100]

'package'

In [199]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

First Original Sentence  Send email to Haris that call me at 5 PM on 12th October
First Encoded Sentence  [4, 3, 1, 53, 2, 33, 24, 9, 25, 26, 7, 54, 34]
Is Length of Original Sentence Same as Encoded Sentence  True
Length of First Sentence  13


In [200]:
print(len(list(set(data['Tag'].values))))

18


In [201]:
tagset = set()
for x in data['Tag']:
    for y in x.split():
        tagset.add(y)
print(len(list(tagset)))

4


In [202]:
tags=list(tagset)
#print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

Number of Tags  4
Tags Map  {'B-email.to': 0, 'O': 1, 'B-email.text': 2, 'I-email.text': 3}


In [203]:
reverse_tag_map={v: k for k, v in tags_map.items()}

In [204]:
print("Reverse Tags Map ", reverse_tag_map)

Reverse Tags Map  {0: 'B-email.to', 1: 'O', 2: 'B-email.text', 3: 'I-email.text'}


In [205]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence ",sentences_list[0])
print('First Sentence Original Tags ',tags_list[0])
print("First Sentence Encoded Tags ",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence ",len(encoded_tags[0]))

First Sentence  Send email to Haris that call me at 5 PM on 12th October
First Sentence Original Tags  ['O', 'O', 'O', 'B-email.to', 'O', 'B-email.text', 'I-email.text', 'I-email.text', 'I-email.text', 'I-email.text', 'I-email.text', 'I-email.text', 'I-email.text']
First Sentence Encoded Tags  [1, 1, 1, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3]
Is length of Original Tags and Encoded Tags same  True
Length of Tags for First Sentence  13


In [206]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

20


In [207]:
max_len=32
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['O'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

Shape of Encoded Sentence  (36, 32)
Shape of Encoded Labels  (36, 32)
First Encoded Sentence Without Padding  [4, 3, 1, 53, 2, 33, 24, 9, 25, 26, 7, 54, 34]
First Encoded Sentence with padding  [ 4  3  1 53  2 33 24  9 25 26  7 54 34  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
First Sentence Encoded Label without Padding  [1, 1, 1, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3]
First Sentence Encoded Label with Padding  [1 1 1 0 1 2 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [208]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence  (32, 4)


In [209]:
target[0]

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [210]:
from sklearn.model_selection import train_test_split
X_train,X_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentences,target,test_size = 0.3,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_val_test,y_val_test,test_size = 0.2,random_state=42)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

Input Train Data Shape  (25, 32)
Train Labels Length  25
Input Test Data Shape  (3, 32)
Test Labels Length  3
Input Validation Data Shape  (8, 32)
Validation Labels Length  8


In [211]:
print("First Sentence in Training Data ",X_train[0])
print("First sentence Label ",y_train[0])
print("Shape of First Sentence -Train",X_train[0].shape)
print("Shape of First Sentence Label  -Train",y_train[0].shape)

First Sentence in Training Data  [  6  12  13   1  14   1 186   2   6  49 187   5  51  52 188  50   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
First sentence Label  [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
Shape of First Sentence -Train (32,)
Shape of First Sentence Label  -Train (32, 4)


In [212]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional

In [240]:
embedding_dim=64
vocab_size=len(tokeniser.word_index)+1
lstm_units=64
max_len=32

input_word = Input(shape = (max_len,))
model = Embedding(input_dim = vocab_size+1,output_dim = embedding_dim,input_length = max_len)(input_word)
model = Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat')(model)
model = LSTM(units=embedding_dim,return_sequences=True)(model)
out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 32, 64)            12480     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 32, 128)           66048     
_________________________________________________________________
lstm_12 (LSTM)               (None, 32, 64)            49408     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 32, 4)             260       
Total params: 128,196
Trainable params: 128,196
Non-trainable params: 0
_________________________________________________________________


In [241]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [242]:
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 32,epochs = 1)

1/1 [==============================] - 9s 9s/step - loss: 1.3609 - accuracy: 0.6963 - val_loss: 1.3283 - val_accuracy: 0.6875


In [243]:
preds=model.predict(X_test) ## Predict using model on Test Data

In [244]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_input']=test_data[i]
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]
    
    return pred_data,accuracy

In [245]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

Shape of Test Data Array (3, 32)
Number of Test Data Points  3


In [246]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [247]:
print(classification_report(y_actual,y_pred))

              precision    recall  f1-score   support

B-email.text       0.17      0.33      0.22         3
  B-email.to       0.00      0.00      0.00         3
I-email.text       0.00      0.00      0.00        22
           O       0.29      0.83      0.43        12

    accuracy                           0.28        40
   macro avg       0.12      0.29      0.16        40
weighted avg       0.10      0.28      0.15        40



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [248]:
pred_data[pred_data['actual_target_tag']=="B-email.to"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
3,174,174,Ibrahim,0,1,B-email.to,O,1
3,189,189,Praveen,0,2,B-email.to,B-email.text,2
3,111,111,Kyle,0,1,B-email.to,O,3


In [249]:
pred_data[pred_data['actual_target_tag']=="B-email.text"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
5,175,175,The,2,1,B-email.text,O,1
5,190,190,Kanchan,2,2,B-email.text,B-email.text,2
5,32,32,Lets,2,1,B-email.text,O,3


In [250]:
y_pred[0]

'O'

In [251]:
y_actual[0]

'O'

In [252]:
pred_data['tokens'][1]

1    email
1    email
1    email
Name: tokens, dtype: object

In [253]:
X_test[0]

array([  8,   3,   1, 174,   2, 175, 176,  23, 177, 178,   5,  47,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)

In [254]:
test_sentences = tokeniser.sequences_to_texts(X_test)

In [255]:
test_sentences[1]

'Send email to Praveen that Kanchan is standing in the Park and I am on my way'

In [256]:
y_test[0]

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [257]:
preds[0]

array([[0.24948406, 0.25111702, 0.25018132, 0.24921764],
       [0.24872527, 0.25213873, 0.2506164 , 0.24851954],
       [0.24819198, 0.2537717 , 0.25033686, 0.24769948],
       [0.24770704, 0.25489473, 0.2501727 , 0.24722552],
       [0.24695376, 0.2565023 , 0.24968748, 0.2468565 ],
       [0.24643986, 0.2579138 , 0.24932304, 0.24632329],
       [0.2458381 , 0.25894797, 0.24927741, 0.24593653],
       [0.24554966, 0.2595634 , 0.24910901, 0.24577795],
       [0.24567032, 0.26104906, 0.24826412, 0.24501647],
       [0.24574874, 0.2624766 , 0.24756452, 0.24421012],
       [0.24575646, 0.2638095 , 0.24697512, 0.24345896],
       [0.24629569, 0.2654404 , 0.24589343, 0.24237047],
       [0.24639344, 0.26785097, 0.24460007, 0.24115556],
       [0.2460482 , 0.27018327, 0.24375616, 0.24001239],
       [0.24545878, 0.27240095, 0.24320072, 0.23893958],
       [0.24475501, 0.27448118, 0.24282518, 0.23793863],
       [0.24401735, 0.27640885, 0.24255918, 0.23701462],
       [0.2432946 , 0.27817303,

In [258]:
pred_data['pred_target_index'][0]

0    1
0    1
0    1
Name: pred_target_index, dtype: int64

In [259]:
pred_data['test_input'][0]

0    8
0    4
0    4
Name: test_input, dtype: int32

In [260]:
pred_data.shape

(40, 8)

In [261]:
X_test.shape

(3, 32)

In [262]:
fin_data = pred_data.groupby('id')

In [263]:
fin_data.head()

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,8,8,Please,1,1,O,O,1
1,3,3,email,1,1,O,O,1
2,1,1,to,1,1,O,O,1
3,174,174,Ibrahim,0,1,B-email.to,O,1
4,2,2,that,1,1,O,O,1
0,4,4,Send,1,1,O,O,2
1,3,3,email,1,2,O,B-email.text,2
2,1,1,to,1,1,O,O,2
3,189,189,Praveen,0,2,B-email.to,B-email.text,2
4,2,2,that,1,2,O,B-email.text,2


In [265]:
pred_data[pred_data['id']==2]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,4,4,Send,1,1,O,O,2
1,3,3,email,1,2,O,B-email.text,2
2,1,1,to,1,1,O,O,2
3,189,189,Praveen,0,2,B-email.to,B-email.text,2
4,2,2,that,1,2,O,B-email.text,2
5,190,190,Kanchan,2,2,B-email.text,B-email.text,2
6,23,23,is,3,2,I-email.text,B-email.text,2
7,191,191,standing,3,2,I-email.text,B-email.text,2
8,31,31,in,3,1,I-email.text,O,2
9,5,5,the,3,1,I-email.text,O,2
